In [6]:
# https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

import pandas as pd
import nltk
import json
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# analyzer = SentimentIntensityAnalyzer()
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer

# download nltk corpus (first time only)
# nltk.download("all")

# from nltk.stem import WordNetLemmatizer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

## Helper functions
### NLTK Application

In [15]:
def merge_columns_in_dataframe(df):
    columns_to_merge = [
        "pagetitle",
        "subtitle",
        "introduction",
        "summarybox",
        "content",
        "accordion",
    ]
    new_df = df.loc[:, ["url", "pagetitle"]].copy()
    new_df["text"] = df.apply(
        lambda row: " ".join([str(row[col]) for col in columns_to_merge]), axis=1
    )
    return new_df

In [4]:
def nltk_text_preprocessing(text):
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import SnowballStemmer

    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german")
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

def get_sentiment(text_string):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()

    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

In [3]:
def add_sentiment_cols(dataframe, analysis_method, text_col="text"):
    if analysis_method == "distilbert":
        sentiment_cols = dataframe[text_col].apply(get_sentiment_distilbert)
    elif analysis_method == "nltk" or analysis_method == "spacy":
        sentiment_cols = dataframe[text_col].apply(get_sentiment)
    else:
        "please provide method (nltk/spacy/distilbert)"
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    # dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [14]:
# def get_analysis_data(json_file):
#     pandas_df = pd.read_json(json_file)
#     filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'pagetitle'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "pagetitle" haben 
#     data = merge_columns_in_dataframe(filtered_df)
#     data["text"] = data["text"].apply(preprocess_text)
#     return add_sentiment_cols(data)

In [2]:
def get_analysis_data(json_file, analysis_method):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'pagetitle'])] != "").any(axis=1)] # excludes all columns from the analysis that have "" in the columns except "pagetitle" 
    data = merge_columns_in_dataframe(filtered_df)
    if analysis_method == "nltk":
        data["text"] = data["text"].apply(nltk_text_preprocessing)
    elif analysis_method == "spacy":
        data["text"] = data["text"].apply(spacy_text_preprocessing)
    return add_sentiment_cols(data, analysis_method)

### SpaCy Lemmatization

In [7]:
def spacy_text_preprocessing(text):
    import spacy
    nlp = spacy.load("de_core_news_sm")

    doc = nlp(text.lower())
    filtered_tokens = [token for token in doc if not token.is_stop] # exclusion of stop words

    lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
    processed_text = " ".join(lemmatized_tokens)
    return processed_text

### Huggingface DistillBERT Model Application

In [1]:
def calculate_average_scores(sentiment_results):
    scores = {'negative': [], 'neutral': [], 'positive': []}
    for result in sentiment_results:
        for res in result:
            scores[res['label']].append(res['score'])
    average_scores = {label: sum(score_list) / len(score_list) if score_list else 0 for label, score_list in scores.items()}
    return average_scores


def get_sentiment_distilbert(text_string):
    from transformers import pipeline
    import spacy

    classifier = pipeline('sentiment-analysis',
        model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
        top_k=None,
        truncation=False,
    )

    try:
        nlp = spacy.load('de_core_news_sm')
        doc = nlp(text_string)

        tokens = [token.text for token in doc]
        results = []
        if len(tokens) > 300:
            for sentence in doc.sents:
                result = classifier(sentence.text)
                results.extend(result)
        else:
            result = classifier(text_string)
            results.extend(result)

        average_scores = calculate_average_scores(results)
        return pd.Series(average_scores)
    except Exception as e:
        print(f"Error processing text: {text_string} with error {e}")
        return pd.Series({'negative': 0, 'neutral': 0, 'positive': 0})

In [17]:
def save_as_json(pandas_dataframe, file_name):
    import datetime
    date_info = datetime.datetime.now().strftime("%d-%m-%y")
    pandas_dataframe.to_json(f"{file_name}_{date_info}.json", orient='records')

In [10]:
# df = get_analysis_data("ratgeber_pages_part2.json")
# save_as_json(df, "ratgeber_pages_with_rating")

## Execution of Code

In [16]:
distilbert_results = get_analysis_data("../web-crawler/scrapy_mobiliar/mobiscraper/mobiscraper/spiders/scrape_archive/full_scrape_III_with_linkedpages.json", "distilbert")
distilbert_results

Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


Error processing text: Fahrsicherheitstraining Beiträge an WAB-Kurs und Fahrsicherheitstraining Wer Gefahrensituationen richtig einschätzt, fährt sicherer. Wir belohnen Ihren Einsatz für sicheres Fahren mit Beiträgen an ein freiwilliges Fahrsicherheitstraining. Inhaber:innen eines Führerausweises auf Probe profitieren von einer Prämienrückvergütung für den obligatorischen Weiterbildungskurs (WAB-Kurs). Vorteile der Beiträge an Ihr Fahrsicherheitstraining auf einen Blick Prämienrabatt bei Absolvierung eines von der Mobiliar anerkannten freiwilligen Fahrsicherheitstrainings Einmaliger Beitrag an freiwilliges Fahrsicherheitstraining Für Personenwagen und Motorräder Prämienrückvergütung beim Besuch des obligatorischen Weiterbildungskurses (WAB)  Welche Beiträge leisten wir an das Fahrsicherheitstraining? Bei einer Motorfahrzeugversicherung der Mobiliar sind folgende Beiträge an Ihr Fahrsicherheitstraining inbegriffen: Prämienrabatt Prämienrabatt für Personenwagen und Motorräder auf Ihre Mo

Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors


Error processing text: Berufs- und Betriebshaftpflicht Schützen Sie sich und Ihre Firma vor den Ansprüchen Dritter Jedes Unternehmen kann Dritte schädigen und so haftpflichtig werden. Eine Betriebs-Haftpflichtversicherung oder eine Berufs-Haftpflichtversicherung entschädigt begründete Ansprüche und wehrt unbegründete Ansprüche ab. Drei Beispiele aus unserer Praxis Eine Firma muss ein Produkt wegen eines Sicherheitsmangels zurückrufen. Wir unterstützen die Firma in der Rückrufaktion und übernehmen alle Kosten. Auf dem Firmengelände rutschen einige Fussgänger und Velofahrer auf Glatteis aus, weil der Firmenparkplatz nur ungenügend gesalzen wurde. Wir decken alle Kosten für die Folgen des mangelhaften Unterhalts. Aus einer Fabrik entweicht nachts durch einen defekten Kaminfilter Aschestaub. Wir regeln den Schaden mit allen Nachbarn und benachbarten Betrieben direkt und unbürokratisch. Die Betriebshaftpflicht Die Betriebs-Haftpflichtversicherung schützt Sie vor den finanziellen Folgen von 

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


Error processing text: Partnerschaftlich und verantwortungsvoll  Umsichtig, unabhängig und kapitalstark betreibt die Mobiliar das Versicherungs- und Anlagegeschäft mit langfristiger Optik und beteiligt anstelle von Aktionären ihre Kundinnen und Kunden am Geschäftserfolg.  Die Mobiliar wurde 1826 als erste private Versicherung der Schweiz und als Genossenschaft gegründet. Diese Rechtsform hat sich für die Mobiliar bewährt. Heute besteht eine schlagkräftige Organisation unter dem Dach einer Genossenschaft. Folgerichtig ist die wirtschaftliche Leistung nach wie vor gezielt auf die Kunden ausgerichtet, die – anstelle von Dividenden an Aktionäre – freiwillige Überschussbeteiligungen erhalten. Die ausgeprägte Kundenorientierung und Kundennähe pflegt die Mobiliar über ihr flächendeckendes Netz von 79 Generalagenturen, die an rund 160 Standorten die kundennahe und persönliche Betreuung sicherstellen, sowie über ihre digitalen Kanäle und Plattformen. Unser Anspruch ist es, gegenüber den Kunden 

,url,pagetitle,text,negative,neutral,positive
0,https://www.mobiliar.ch,Versicherungen & Vorsorge,Versicherungen & Vorsorge,0.230486,0.140164,0.629349
1,https://www.mobiliar.ch/versicherungen-und-vor...,Tierversicherung,Tierversicherung Hunde und Katzen versichern D...,0.458895,0.168780,0.372325
2,https://www.mobiliar.ch/versicherungen-und-vor...,Cyberversicherung,Cyberversicherung Hilfe im digitalen Alltag Ba...,0.431041,0.155108,0.413851
3,https://www.mobiliar.ch/versicherungen-und-vor...,Wohnen & Eigentum,Wohnen & Eigentum,0.208000,0.195661,0.596339
4,https://www.mobiliar.ch/versicherungen-und-vor...,Mietkautionsversicherung,Mietkautionsversicherung Dank einer Mietkautio...,0.338874,0.185144,0.475982
...,...,...,...,...,...,...
1739,https://www.mobiliar.ch/die-mobiliar/ueber-uns...,Wir befähigen für die Zukunft,Wir befähigen für die Zukunft Von der Idee zur...,0.228822,0.119091,0.652087
1740,https://www.mobiliar.ch/die-mobiliar/ueber-uns...,Wir denken Zukunft,Wir denken Zukunft Erfahrungen und Erkenntniss...,0.181774,0.126676,0.691551
1741,https://www.mobiliar.ch/die-mobiliar/ueber-uns...,Leitbild der Gruppe Mobiliar,Leitbild der Gruppe Mobiliar \n\tDas Leitbild...,0.245275,0.097418,0.657307
1742,https://www.mobiliar.ch/die-mobiliar/ueber-uns...,Geisterhaus-Spot gewinnt Silber-Edi.,Geisterhaus-Spot gewinnt Silber-Edi. \n\tAusz...,0.337304,0.129163,0.533533


In [18]:
# save_as_json(pandas_dataframe=distilbert_results, file_name="distilbert_s_a_result")

In [20]:
get_analysis_data("example.json", "distilbert")

,url,pagetitle,text,negative,neutral,positive
0,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,0.067596,0.063357,0.869048
1,https://www.mobiliar.ch/versicherungen-und-vor...,Gebäudeversicherung – der Rundumschutz für Ihr...,Gebäudeversicherung – der Rundumschutz für Ihr...,0.430694,0.165596,0.403710
2,https://www.mobiliar.ch/versicherungen-und-vor...,E-Autos laden: Was Sie alles wissen müssen,E-Autos laden: Was Sie alles wissen müssen Ele...,0.346649,0.205312,0.448039
3,https://www.mobiliar.ch/die-mobiliar/medien/me...,Digitale Identität: Mobiliar beteiligt sich an...,Digitale Identität: Mobiliar beteiligt sich an...,0.359737,0.110319,0.529944
4,https://www.mobiliar.ch/die-mobiliar/karriere/...,System Engineering,System Engineering Begleiten Sie die Mobiliar ...,0.256660,0.135608,0.607732


In [21]:
get_analysis_data("example.json", "spacy")

,url,pagetitle,text,neg,neu,pos,compound
0,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,Kunst & Nachhaltigkeit vol -- 14 -- 25 prix Mo...,0.000,0.989,0.011,0.2263
1,https://www.mobiliar.ch/versicherungen-und-vor...,Gebäudeversicherung – der Rundumschutz für Ihr...,Gebäudeversicherung -- Rundumschutz Haus finan...,0.005,0.988,0.007,0.0516
2,https://www.mobiliar.ch/versicherungen-und-vor...,E-Autos laden: Was Sie alles wissen müssen,e-autos laden -- wissen Elektroauto Vormarsch ...,0.000,0.990,0.010,0.3612
3,https://www.mobiliar.ch/die-mobiliar/medien/me...,Digitale Identität: Mobiliar beteiligt sich an...,digital Identität -- Mobiliar beteiligen gemei...,0.000,0.958,0.042,0.3818
4,https://www.mobiliar.ch/die-mobiliar/karriere/...,System Engineering,System Engineering begleiten Mobiliar Weiteren...,0.005,0.967,0.028,0.9531


In [22]:
get_analysis_data("example.json", "nltk")

,url,pagetitle,text,neg,neu,pos,compound
0,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Kunst & Nachhaltigkeit Vol. 14: 25 Jahre Prix ...,kunst & nachhalt vol . 14 : 25 jahr prix mobil...,0.000,0.987,0.013,0.2263
1,https://www.mobiliar.ch/versicherungen-und-vor...,Gebäudeversicherung – der Rundumschutz für Ihr...,gebaudeversicher – rundumschutz haus finanziel...,0.006,0.982,0.012,0.2869
2,https://www.mobiliar.ch/versicherungen-und-vor...,E-Autos laden: Was Sie alles wissen müssen,e-autos lad : wiss muss elektroautos vormarsch...,0.014,0.951,0.035,0.6767
3,https://www.mobiliar.ch/die-mobiliar/medien/me...,Digitale Identität: Mobiliar beteiligt sich an...,digital identitat : mobiliar beteiligt gemeins...,0.000,0.951,0.049,0.3818
4,https://www.mobiliar.ch/die-mobiliar/karriere/...,System Engineering,syst engineering begleit mobiliar weiterentwic...,0.013,0.970,0.018,0.5994


In [ ]:
# df2 = get_analysis_data("../web-crawler/scrapy_mobiliar/mobiscraper/mobiscraper/spiders/scrape_archive/full_scrape_2.json")
# save_as_json(df2, "full_scrape_2_analysis")

In [12]:
df2 = pd.read_json("full_scrape_2_analysis_15-03-24_15:40:08.json")
display(df2)

,url,page_title,text,neg,neu,pos,compound,judgement
0,https://www.mobiliar.ch/versicherungen-und-vor...,Tierversicherung,tierversicher hund katz versich tierversicher ...,0.000,0.994,0.006,0.2406,positive
1,https://www.mobiliar.ch/versicherungen-und-vor...,Cyberversicherung,cyberversicher hilf digital alltag bankgeschaf...,0.028,0.954,0.019,-0.5362,negative
2,https://www.mobiliar.ch/versicherungen-und-vor...,Wertsachenversicherung,"wertsachenversicher versich , bedeutet wertvol...",0.000,0.981,0.019,0.6553,positive
3,https://www.mobiliar.ch/versicherungen-und-vor...,Die Hausratversicherung,hausratversicher rundum-versicher beweg sach w...,0.021,0.979,0.000,-0.6662,negative
4,https://www.mobiliar.ch/versicherungen-und-vor...,Mietkautionsversicherung,mietkautionsversicher dank mietkaution spar mi...,0.007,0.976,0.017,0.4329,positive
...,...,...,...,...,...,...,...,...
1456,https://www.mobiliar.ch/versicherungen-und-vor...,Die Fahrrad-Kaskoversicherung vermitteln,fahrrad-kaskoversicher vermitteln umfass versi...,0.028,0.972,0.000,-0.3313,negative
1457,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2022: Insgesamt 19 weitere Brücken un...,projekt 2022 : insgesamt 19 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1458,https://www.mobiliar.ch/die-mobiliar/nachhalti...,Projekte 2023: Insgesamt 13 weitere Brücken un...,projekt 2023 : insgesamt 13 weit bruck steg fe...,0.000,1.000,0.000,0.0000,neutral
1459,https://www.mobiliar.ch/versicherungen-und-vor...,Winter Wanderland,"wint wanderland verschneit tal , weit nebelme ...",0.000,1.000,0.000,0.0000,neutral


In [13]:
df2["judgement"].value_counts()

judgement
neutral     726
positive    465
negative    270
Name: count, dtype: int64

## to display the Functions

In [38]:
text = "Der neue «Liebe Mobiliar»-Spot zeigt, dass Schäden zu Hause immer genau dann passieren, wenn man nicht damit rechnet.  Die Mobiliar startet mit dem neusten «Liebe Mobiliar»-Spot ins neue Jahr. Die Familie im Spot hingegen startet in einen nur auf den ersten Blick gewöhnlichen Sonntagmorgen. Denn: Während die Eltern noch gemütlich vor sich hinschlummern, werden sie von Geräuschen geweckt, die definitiv nicht ins Haus gehören. Grund dafür ist ihre Tochter, die ihr Talent als Eiskunstläuferin entdeckt und mit viel Freude auslebt. Und das hinterlässt Spuren. Was immer kommt – wir helfen Ihnen rasch und unkompliziert."

In [31]:
text = "Heute war ein wunderschöner Tag. Ich ging in den Wald spazieren und habe die Tiere beobachtet. Ich freue mich dies bald wieder zu tun."


In [26]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(input_text.lower())
output_text = " ".join(tokens)
output_text

NameError: name 'input_text' is not defined

In [39]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text.lower())
output_text = " ".join(tokens)
output_text

'der neue « liebe mobiliar » -spot zeigt , dass schäden zu hause immer genau dann passieren , wenn man nicht damit rechnet . die mobiliar startet mit dem neusten « liebe mobiliar » -spot ins neue jahr . die familie im spot hingegen startet in einen nur auf den ersten blick gewöhnlichen sonntagmorgen . denn : während die eltern noch gemütlich vor sich hinschlummern , werden sie von geräuschen geweckt , die definitiv nicht ins haus gehören . grund dafür ist ihre tochter , die ihr talent als eiskunstläuferin entdeckt und mit viel freude auslebt . und das hinterlässt spuren . was immer kommt – wir helfen ihnen rasch und unkompliziert .'

In [40]:
from nltk.corpus import stopwords
filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
output_text = " ".join(filtered_tokens)
output_text

'neue « liebe mobiliar » -spot zeigt , schäden hause immer genau passieren , rechnet . mobiliar startet neusten « liebe mobiliar » -spot neue jahr . familie spot hingegen startet ersten blick gewöhnlichen sonntagmorgen . : eltern gemütlich hinschlummern , geräuschen geweckt , definitiv haus gehören . grund dafür tochter , talent eiskunstläuferin entdeckt freude auslebt . hinterlässt spuren . immer kommt – helfen rasch unkompliziert .'

In [41]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("german")
stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
output_text_nltk = " ".join(stemmed_tokens)
output_text_nltk

'neu « lieb mobiliar » -spot zeigt , schad haus imm genau passi , rechnet . mobiliar startet neust « lieb mobiliar » -spot neu jahr . famili spot hingeg startet erst blick gewohn sonntagmorg . : elt gemut hinschlumm , gerausch geweckt , definitiv haus gehor . grund dafur tocht , talent eiskunstlauferin entdeckt freud auslebt . hinterlasst spur . imm kommt – helf rasch unkompliziert .'

In [42]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = analyzer.polarity_scores(output_text_nltk)
pd.Series(scores)

neg         0.0000
neu         0.9460
pos         0.0540
compound    0.4215
dtype: float64

In [43]:
import spacy
nlp = spacy.load("de_core_news_sm")

doc = nlp(text.lower())
filtered_tokens = [token for token in doc if not token.is_stop] # exclusion of stop words

lemmatized_tokens = [token.lemma_ for token in filtered_tokens]
output_text_spacy = " ".join(lemmatized_tokens)
output_text_spacy

'-- liebe mobiliar»-spot zeigen -- schäden Hause genau passieren -- rechnen --   Mobiliar starten neu -- liebe mobiliar»-spot -- Familie Spot hingegen starten Blick gewöhnlich Sonntagmorg -- -- Eltern gemütlich hinschlummern -- geräuschen wecken -- definitiv Haus gehören -- Grund Tochter -- Talent eiskunstläuferin entdecken Freude ausleben -- Hinterlässt Spur -- -- helfen rasch unkompliziert --'

In [44]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = analyzer.polarity_scores(output_text_spacy)
pd.Series(scores)

neg         0.0000
neu         0.9520
pos         0.0480
compound    0.4215
dtype: float64

In [85]:
text = "Für welche Schäden haften Sie als Mieterin oder Mieter? Die Freude ist gross: In Ihrer neuen Mietwohnung fühlen Sie sich rundum wohl und wörtlich zu Hause. Doch nach einer Woche wird das Wohnvergnügen etwas getrübt. Sie stellen einen Sprung in der Badewanne fest, den Sie bei der Wohnungsübernahme übersehen haben. Müssen Sie den Schaden nun bezahlen? Als Mieterin oder Mieter haften Sie für Schäden, die Sie selbst, Ihre Mitbewohner, Angestellte, Gäste, Hund und Katze oder andere Tiere verursachen. Schäden, die durch normale Abnützung entstehen, muss der Vermieter übernehmen. Wenn Sie nach Ihrem Einzug Mängel entdecken, die nicht protokolliert wurden, sollten Sie diese mittels Mängelliste Ihrem Vermieter melden. So stellen Sie sicher, dass Sie beim Auszug nicht dafür haften. Sie haben für die Meldung je nach Vermieter 5-30 Tage nach der Wohnungsübergabe Zeit. Für solche Schäden haften Sie Ihr fünfjähriger Sohn hat die Wohnzimmerwand verkritzelt, Ihre Katze entdeckt ihren Jagdinstinkt nach Insekten und zerkratzt dabei die Tapete – und nach einer gründlichen Putzaktion entdecken Sie einen Kratzer im Keramikkochfeld. Solche und ähnliche Schäden müssen Sie selbst bezahlen. Ihre Haftpflichtversicherung übernimmt die Kosten nach Abzug des Selbstbehalts und Amortisationskosten. Für solche Schäden haften Sie nicht Für die normale Abnützung haften Sie nicht: vergilbte Tapeten, Spuren von Bildern und Möbeln an den Wänden, abgenützte Teppiche, sachgerecht verschlossene Nagel- und Dübellöcher, Schäden von unbekannten Dritten. Und natürlich müssen Sie auch keinen Schaden bezahlen, der schon bei Mietantritt bestand und im Abnahmeprotokoll aufgeführt ist. Bei Mietschäden die Versicherung informieren Besprechen Sie jeden Schaden an Ihrer Mietwohnung so rasch wie möglich mit der Hausverwaltung und informieren Sie Ihre Haftpflichtversicherung . Bei einem grösseren Schaden oder wenn die Verwaltung Schadenersatz fordert, sind Sie sogar verpflichtet, Ihre Versicherung zu benachrichtigen. Die Höhe des Schadenersatzes für eine beschädigte Sache richtet sich nach ihrer durchschnittlichen Lebensdauer. Die normale Lebensdauer eines Spannteppichs beträgt zum Beispiel zehn Jahre. Muss er wegen einer Beschädigung bereits nach fünf Jahren erneuert werden, bezahlen Sie höchstens 50 Prozent des Neuwerts. Kleine Reparaturen und Ausbesserungen Wenn die Reparatur kleiner Schäden unverhältnismässig teuer wäre, müssen Sie nur einen Teil bezahlen. So darf Ihr Vermieter wegen eines kleinen Brandlochs im Spannteppich nicht den ganzen Teppich auf Ihre Kosten ersetzen. Eine Reparatur der Stelle genügt. Diese übernimmt Ihre Haftpflichtversicherung nach Abzug des Selbstbehalts. Als Mieter oder Mieterin müssen Sie folgende Reparaturen und Ausbesserungen an Ihrer Wohnung selbst bezahlen: Instandstellung von Schaltern, Steckdosen und Türschlössern Ersatz von Hahnendichtungen und Duschschläuchen Reinigung und Ersatz von Küchenabluft-Filtern Entrussen von Wohnzimmer-Cheminées Auswechseln von Sicherungen Entstopfen von Ablaufrohren bis zur Hauptleitung Beheben Sie grössere Schäden nicht selbst Vergeben Sie Reparaturaufträge nie direkt, sondern benachrichtigen Sie Ihre Hausverwaltung. Sie haben sonst keinen Anspruch darauf, dass Ihnen die Verwaltung jenen Teil der Arbeit, der auf die Altersentwertung zurückzuführen ist, vergütet. Wenn Ihre Vermieterin zudem unzufrieden mit der Ausführung ist und die Reparatur nicht akzeptiert, müssen Sie doppelt bezahlen. Schäden rechtzeitig reparieren Bei Mietbeginn müssen alle kleinen Reparaturen ausgeführt sein. Kontaktieren Sie Ihre Hausverwaltung frühzeitig, wenn Sie ausziehen und grössere Reparaturen auf Ihre Kosten ausgeführt werden müssen. So lässt sich der Schaden beheben, bevor die Nachmieterin oder der Nachmieter einzieht. Sollte sich nämlich ihr Einzug wegen der Reparatur verzögern, müssen Sie den Mietzinsausfall und weitere Umtriebe bezahlen. Entdecken Sie hier weitere Tipps rund ums Thema Umzug ! Durchschnittliche Lebensdauer von Wohnungseinrichtungen Decken, Wände, Türen Tapeten, mittlere Qualität (Raufaser, bedruckt oder roh gestrichen) 10 Jahre Tapeten zum Überstreichen 24 Jahre Anstrich mit Kunstharz, Acryl, Alkydharzfarben 15 Jahre Anstrich mit Dispersions-, Leimfarbanstrich, Blanc-fix 8 Jahre Holzverkleidung 30 Jahre Holztäfer (deckend gestrichen) 30 Jahre Türrahmen / Türschwellen (innen, Holz) 30 Jahre Fenstersimse (innen, lackiert mit Öl-, Acryl- oder Kunstharzfarbanstrich) 20 Jahre Holz- / Täferdecken 40 Jahre Keramikplatten 30 Jahre Bodenbeläge Novilon etc. 10 Jahre Linoleum 20 Jahre Laminat, schlechte Qualität 10 Jahre Laminat, mittlere Qualität 15 Jahre Laminat, gehobene Qualität 25 Jahre Parkett 30 - 40 Jahre Versiegelung des Parketts 10 Jahre Fournierparkett 10 Jahre Kunststeinplatten 40 Jahre Spannteppich, mittlere Qualität 10 Jahre Naturfaserteppiche / Sisal-Kokos 10 Jahre Nadelfilz 8 Jahre Sockel Kunststoff 15 Jahre Holz 25 Jahre Storen, Rolläden Storenstoff 15 Jahre Rollläden aus Holz 25 Jahre Rolläden aus Metall 30 Jahre Lamellenstoren innen aus Kunststoff oder Aluminium 15 Jahre Lamellenstoren aussen aus Aluminium 25 Jahre Kurbeln 15 Jahre Kurbelhalterung aus Metall 10 Jahre Kurbelhalterung aus Kunststoff 5 Jahre Küche Kochherd / Backofen 15 Jahre Induktionskochfeld / Glaskeramikkochfeld 15 Jahre Elektrische Herdplatte 15 Jahre Kühlschrank 10 Jahre Tiefkühler 15 Jahre Geschirrspüler 15 Jahre Dampfabzug, inkl. Metallfilter 10 Jahre Mikrowelle 15 Jahre Steamer 10 Jahre Bedienungsarmaturen 20 Jahre Küchenabdeckung aus Chromstahl, Granit, Glas 25 Jahre Küchenabdeckung, Kunstharzoberfläche 15 Jahre Küchenabdeckung, Holz massiv oder Mehrschichtplatte 20 Jahre Bad,  Dusche,  WC Badewanne / Duschetasse aus Acryl 25 Jahre Badewanne / Duschetasse aus Stahl, emailliert 35 Jahre Emaillierung der Badewanne / Duschetasse 20 Jahre Lavabo / Bidet / WC aus Keramik 35 Jahre Duschkabine aus Kunststoff 15 Jahre Duschkabine mit Glaswänden 25 Jahre Badezimmermöbel aus Kunststoff oder Holz 10 Jahre Seifenschalen- und Zahnglashalter, verchromt 15 Jahre Handtuch- und Haltestangen, verchromt 15 Jahre Vorhangstangen, verchromt 10 Jahre Glastablar 15 Jahre Spiegel 20 Jahre Wandplatten aus Keramik, lasiert 30 Jahre Wandplatten aus Steingut oder Glasmosaik (eingefärbt) 30 Jahre Fugendichtungen / Kittfugen 8 Jahre Elektrische Installationen TV-Kabelanschluss 15 Jahre Steckdosen 15 Jahre Decken- und Wandleuchten in Küche, Bad, WC 20 Jahre Diverses Waschmaschine / Tumbler 15 Jahre Cheminée / Cheminéeofen 25 Jahre Reduktion bei besonderer Beanspruchung Büros 20% Läden 25% Restaurants 50%"

In [86]:
from transformers import pipeline
import spacy

classifier = pipeline('sentiment-analysis',
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    top_k=None,
    truncation=False,
)

nlp = spacy.load('de_core_news_sm')
doc = nlp(text)
tokens = [token.text for token in doc]
results = []

if len(tokens) > 512:
    for sentence in doc.sents:
        result = classifier(sentence.text)
        results.extend(result)
else:
    result = classifier(text)
    results.extend(result)


average_scores = calculate_average_scores(results)
pd.Series(average_scores)

negative    0.414906
neutral     0.181137
positive    0.403957
dtype: float64

In [31]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis',
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    top_k=None,
    truncation=False,
)
result_in_list_list_of_dicts_of_tuples = classifier(text)
mydict = {}
for result_in_list_of_dicts_of_tuples in result_in_list_list_of_dicts_of_tuples:
    for dicts in result_in_list_of_dicts_of_tuples:
        mydict[dicts["label"]] = dicts["score"]
pd.Series(mydict)

#ausgegeben werden confidenzscores 

negative    0.642108
positive    0.272815
neutral     0.085077
dtype: float64

## Training the Model
Habe ich nicht umgesetzt.

In [5]:
from transformers import AutoTokenizer
from transformers import pipeline
import torch.nn.functional as F
import torch

model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
classifier = pipeline(
    model=model_name,
    tokenizer = AutoTokenizer.from_pretrained(model_name),
    top_k=None
)

X_train = ["Ich bin Roman und 28 Jahre alt.", "Ich liebe dich!"]
res = classifier(X_train)

print(res)
tokenizer = AutoTokenizer.from_pretrained(model_name)
batch = tokenizer(X_train, padding=True, truncation=True, max_length=800, return_tensors="pt")

with torch.no_grad():
    outputs = model_name(**batch),
    print(outputs)
    predictions = F.softmax(output.logits, dim=1),
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)


# list_list_of_dicts_of_tuples = res
# for list_of_dicts_of_tuples in list_list_of_dicts_of_tuples:
#     for dicts in list_of_dicts_of_tuples:
#         mydict[dicts["label"]] = dicts["score"]
# print(pd.Series(mydict))

[[{'label': 'positive', 'score': 0.49148350954055786}, {'label': 'neutral', 'score': 0.25997069478034973}, {'label': 'negative', 'score': 0.2485458105802536}], [{'label': 'positive', 'score': 0.9740146994590759}, {'label': 'neutral', 'score': 0.015805009752511978}, {'label': 'negative', 'score': 0.01018031220883131}]]


TypeError: 'str' object is not callable